In [ ]:
import pandas as pd
data=pd.read_csv('E:/作业/杂物/数据集/code_samples.csv')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
vec=CountVectorizer()
x=data.loc[:,'code']
y=data.loc[:,'label']
x=vec.fit_transform(x)
model=RandomForestClassifier()
model.fit(x,y)
y_predict=model.predict(x)
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y,y_predict)
print(accuracy)

In [17]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack, csr_matrix

import joblib
import os  #  加载必要的库

data_path = 'E:/作业/杂物/数据集/111_fixed.csv'
model_path = 'E:/python/models'
chunk_size = 20000
trees = 100
max_depth = 15  #准备好参数


def extract_features(url): #结构化特征，优化使用
    features = {} #创建空字典
    url = str(url) #确保url都是字符串
    features['url_length'] = len(url) # 看长度
    features['count_dots'] = url.count('.') #看点的数量
    features['count_hyphens'] = url.count('-') #看横杠的数量
    suspicious_keywords = ['login', 'secure', 'account', 'verify', 'password', 'update', 'banking'] #看敏感词在恶意网站中的敏感词
    features['count_suspicious_words'] = sum( #这里是数组的原因是我们的url不止一个
        1 for kw in suspicious_keywords if kw in url.lower()
    ) #遍历一遍看有多少敏感词，统计出来个数2
    return features

if not os.path.exists(model_path): #判断文件是否存在，不存在就创建
    os.makedirs(model_path)

print(" 第一步：读取全部数据拟合统一 vectorizer...")
df_all = pd.read_csv(data_path,encoding='ISO-8859-1') #全部数据！！
# 清洗：只保留非空的 URL 和标签
df_all = df_all.dropna(subset=['url', 'label'])
df_all = df_all[df_all['url'].astype(str).str.strip() != '']
xx = pd.DataFrame([extract_features(u) for u in df_all['url']])
vec = TfidfVectorizer() #创建向量器，类似于片段指向的数字这样的转换过程
x_vec=vec.fit_transform(df_all['url'])  #保证训练集每一批是用同一个词袋映射出来的向量，模型每学一批，其实是在同一个特征空间中进化。一致）

scaler = MinMaxScaler()
structured_train_scaled = scaler.fit_transform(xx) #防止数据特征本身太大或太小太影响结果判断转换成介于(0,1)
structured_train_sparse = csr_matrix(structured_train_scaled) #转换成稀疏矩阵进行拼接因为前面 x_vec（TF-IDF 特征）就是稀疏矩阵必须跟它格式一样才能拼接！
x_final = hstack([x_vec, structured_train_sparse]) #合并数据将特征和数据本身进行合并变成一个
y_all = df_all['label']

joblib.dump(vec, os.path.join(model_path, 'vectorizer.pkl')) #保存向量器
joblib.dump(scaler, os.path.join(model_path, 'scaler.pkl'))
print("vectorizer 保存完成！")
print(" 第二步：分批训练模型...")
chunks = pd.read_csv(data_path, chunksize=chunk_size,encoding='ISO-8859-1') #分批读取数据
for i, chunk in enumerate(chunks):
    print(f"\n 正在训练第 {i+1} 批数据（共 {len(chunk)} 条）...")
    chunk = chunk.dropna(subset=['url', 'label']) # 先清理掉空数据
    x = vec.transform(chunk['url'])  # 用统一的 vectorizer 进行 transform
    xx1=pd.DataFrame([extract_features(u) for u in chunk['url']])
    y = chunk['label']

    structured_scaled2 = scaler.transform(xx1) #不用再fit了！！！我们应当使用之前的规则
    structured_sparse2 = csr_matrix(structured_scaled2) 
    x_final2 = hstack([x, structured_sparse2]) 
    
    rf = RandomForestClassifier(n_estimators=trees, max_depth=max_depth, random_state=42) #创建模型
    rf.fit(x_final2, y) #训练模型
    model_filename = os.path.join(model_path, f"rf_model_part_{i+1}.pkl") #得到保存路径
    joblib.dump(rf, model_filename) #按照上边的路径进行保存
    print(f" 模型已保存：{model_filename}")
print("\n 所有模型训练完成！") #模型训练完毕


 第一步：读取全部数据拟合统一 vectorizer...
vectorizer 保存完成！
 第二步：分批训练模型...

 正在训练第 1 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_1.pkl

 正在训练第 2 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_2.pkl

 正在训练第 3 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_3.pkl

 正在训练第 4 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_4.pkl

 正在训练第 5 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_5.pkl

 正在训练第 6 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_6.pkl

 正在训练第 7 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_7.pkl

 正在训练第 8 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_8.pkl

 正在训练第 9 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_9.pkl

 正在训练第 10 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_10.pkl

 正在训练第 11 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_11.pkl

 正在训练第 12 批数据（共 20000 条）...
 模型已保存：E:/python/models\rf_model_part_12.pkl

 所有模型训练完成！


In [19]:
import os
import joblib
import numpy as np
import pandas as pd  # 加载必要的库

def extract_features(url): #结构化特征，优化使用
    features = {} #创建空字典
    url = str(url) #确保url都是字符串
    features['url_length'] = len(url) # 看长度
    features['count_dots'] = url.count('.') #看点的数量
    features['count_hyphens'] = url.count('-') #看横杠的数量
    suspicious_keywords = ['login', 'secure', 'account', 'verify', 'password', 'update', 'banking'] #看敏感词在恶意网站中的敏感词
    features['count_suspicious_words'] = sum( #这里是数组的原因是我们的url不止一个
        1 for kw in suspicious_keywords if kw in url.lower()
    ) #遍历一遍看有多少敏感词，统计出来个数2
    return features

model_path = 'E:/python/models'
model_files = sorted([f for f in os.listdir(model_path) if f.endswith('.pkl') and 'rf_model' in f])
models = [joblib.load(os.path.join(model_path, f)) for f in model_files]
print(f" 加载了 {len(models)} 个模型")

vec = joblib.load('E:/python/models/vectorizer.pkl')

data = pd.read_csv('E:/作业/杂物/数据集/archive/malicious_labeled.csv')
urls = data["url"].tolist()[:10]  # 前 10 条做演示

for idx, url in enumerate(urls):
    sample_vec = vec.transform([url])
    
    if sample_vec.shape[1] == 0:
        print(f"[第 {idx+1} 条] {url} ->  无法向量化，跳过")
        continue

    votes = []
    for i, model in enumerate(models):
        pred = model.predict(sample_vec)[0]
        votes.append(pred)
    
    if not votes:
        print(f"[第 {idx+1} 条] {url} ->  无投票结果")
        continue

    final_result = int(np.round(np.mean(votes)))
    print(f"[第 {idx+1} 条] {url} -> {'恶意' if final_result == 1 else '正常 '}")


 加载了 7 个模型
[第 1 条] br-icloud.com.br -> 正常 
[第 2 条] http://www.garage-pirenne.be/index.php?option=com_content&view=article&id=70&vsig70_0=15 -> 恶意
[第 3 条] http://adventure-nicaragua.net/index.php?option=com_mailto&tmpl=component&link=aHR0cDovL2FkdmVudHVyZS1uaWNhcmFndWEubmV0L2luZGV4LnBocD9vcHRpb249Y29tX2NvbnRlbnQmdmlldz1hcnRpY2xlJmlkPTQ3OmFib3V0JmNhdGlkPTM2OmRlbW8tYXJ0aWNsZXMmSXRlbWlkPTU0 -> 正常 
[第 4 条] http://www.pashminaonline.com/pure-pashminas -> 正常 
[第 5 条] http://www.ikenmijnkunst.nl/index.php/exposities/exposities-2006 -> 正常 
[第 6 条] http://www.lebensmittel-ueberwachung.de/index.php/aktuelles.1 -> 正常 
[第 7 条] http://www.szabadmunkaero.hu/cimoldal.html?start=12 -> 恶意
[第 8 条] http://larcadelcarnevale.com/catalogo/palloncini -> 正常 
[第 9 条] http://www.vnic.co/khach-hang.html -> 正常 
[第 10 条] signin.eby.de.zukruygxctzmmqi.civpro.co.za -> 正常 


In [18]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack, csr_matrix #加载必要的库

def extract_features(url): #结构化特征，优化使用
    features = {} #创建空字典
    url = str(url) #确保url都是字符串
    features['url_length'] = len(url) # 看长度
    features['count_dots'] = url.count('.') #看点的数量
    features['count_hyphens'] = url.count('-') #看横杠的数量
    suspicious_keywords = ['login', 'secure', 'account', 'verify', 'password', 'update', 'banking'] #看敏感词在恶意网站中的敏感词
    features['count_suspicious_words'] = sum( #这里是数组的原因是我们的url不止一个
        1 for kw in suspicious_keywords if kw in url.lower()
    ) #遍历一遍看有多少敏感词，统计出来个数
    return features

model_path = 'E:/python/models'
vec = joblib.load(os.path.join(model_path, 'vectorizer.pkl')) #加载之前的向量器，可以理解成将文本转换成数字使用的是和之前一样的规则，这个向量器就像规则一样

model_files = []
for file in os.listdir(model_path):
    if file.startswith('rf_model_part_') and file.endswith('.pkl'): #加载所有的模型文件
        model_files.append(file)

models = []
for f in model_files:
    models.append(joblib.load(os.path.join(model_path, f))) #加载出所有的模型文件里的模型
print(f"加载了 {len(models)} 个模型")

test_data = pd.read_csv('E:/作业/杂物/数据集/balanced_test_set.csv', encoding='gb18030')  # 加载测试数据集
scaler = joblib.load(os.path.join(model_path, 'scaler.pkl')) 
urls = test_data['url'].tolist() #将数据中的url列单摘出来然后转化成文本
true_labels = test_data['label'].tolist() #加载label列
xx1 = pd.DataFrame([extract_features(u) for u in urls])
x_test = vec.transform(urls)  # 用训练时的 vectorizer.pkl 直接转换
print("向量化完成！") #注意这里不能重新fit，只能 transform（防止特征不一致）

all_votes = []
for i, model in enumerate(models):
    print(f"模型{i+1} 正在预测...")

    structured_scaled2 = scaler.transform(xx1) #不用再fit了！！！我们应当使用之前的规则
    structured_sparse2 = csr_matrix(structured_scaled2) #测试集数据也应该这样做
    x_final2 = hstack([x_test, structured_sparse2]) 

    preds = model.predict(x_final2) #用模型进行预测
    all_votes.append(preds)  # 维度是 (模型数量, 样本数)因为是多个模型预测每个结果都是一个数组所以每一行是每个模型预测的结果列是模型有几个模型就有几行

# 转置一下，得到每个样本的“投票结果”
votes_array = np.array(all_votes).T  # 把他变成一行的二维数组
# 多数投票：每一行求平均，大于0.5算恶意
final_preds = (votes_array.mean(axis=1) >= 0.5).astype(int)
acc = accuracy_score(true_labels, final_preds) #计算准确率
print(f"\n整体准确率为：{acc:.4f}")


加载了 12 个模型
向量化完成！
模型1 正在预测...
模型2 正在预测...
模型3 正在预测...
模型4 正在预测...
模型5 正在预测...
模型6 正在预测...
模型7 正在预测...
模型8 正在预测...
模型9 正在预测...
模型10 正在预测...
模型11 正在预测...
模型12 正在预测...

整体准确率为：0.5000
